In [1]:
# To find values within a column...
#df.loc[df['column_name'] == some_value]
#x = dfs[0][1]
#x.loc[x["ncli"] == 8151]

In [2]:
import os
import re
import math
import pandas as pd
import numpy as np

import plotly
import plotly.plotly as py
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='theano', api_key='hkYRYxjyM0aAGA5nmGq1')

In [3]:
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

In [4]:
home = os.getcwd()
print(home)
sorted([x for x in os.listdir(home)])
collectl_stats = ['Date', '[CPU]User%', '[CPU]Nice%', '[CPU]Sys%', '[CPU]Wait%',
                               '[CPU]Irq%', '[CPU]Soft%', '[CPU]Steal%', '[CPU]Idle%',
                               '[CPU]Totl%', '[CPU]Intrpt/sec', '[CPU]Ctx/sec', '[CPU]Proc/sec',
                               '[CPU]ProcQue', '[CPU]ProcRun', '[CPU]L-Avg1', '[CPU]L-Avg5',
                               '[CPU]L-Avg15', '[CPU]RunTot', '[CPU]BlkTot']
loaded_files = []


/home/theano/mountpoint/tsch/mininetpipe/results/2017-12-14-08-59


In [5]:
def load_datfile(fpath):
    # Returns a dataframe
    df = pd.read_csv(fpath, 
                     index_col=False,
                     names=["ncli","throughput"])
    return df


def load_collectl(fname):
    df = pd.read_csv(fname,
                     delim_whitespace=True,
                     comment="#",
                     names=collectl_stats)
    return df

def load_data_and_stats(basename):
    pds = []
    prog = re.compile("%s_u\d+_\d+.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        print(fname)
        df_tput = load_datfile(os.path.join(home, fname))
        
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])

        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
            
        newdf = pd.concat([df_tput.sum(), df_stats.mean()])
        newdf["fname"] = fname
        newdf["expduration"] = expduration
        newdf["ncores"] = int(fname.split("cores")[0])        
        pds.append(newdf)
        
    return (basename, pd.concat(pds, axis=1).transpose())

def load_data_no_sum(basename):
    pds = []
    prog = re.compile("%s_u\d+_\d+.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
        
        newdf = pd.concat([df_tput, df_stats])
        newdf["expduration"] = expduration
        newdf["fname"] = fname  # for finding weird data
        newdf["ncores"] = int(fname.split("cores")[0])
        pds.append(newdf)

    return (basename, pd.concat(pds).apply(pd.to_numeric, errors='ignore'))

In [45]:
df1 = load_data_and_stats("32cores")
df2 = load_data_and_stats("16cores_on")
df3 = load_data_and_stats("8cores")

OSError: [Errno 107] Transport endpoint is not connected: '.'

In [ ]:
df1 = (df1[0], df1[1].loc[df1[1]["ncli"] == 2048])
df2 = (df2[0], df2[1].loc[df2[1]["ncli"] == 2048])
df3 = (df3[0], df3[1].loc[df3[1]["ncli"] == 2048])

dfs = [df1, df2, df3]

In [ ]:
grouped = pd.concat([df.apply(pd.to_numeric, errors='coerce') for name, df in dfs]).groupby("ncores")
agged = grouped.agg([np.std, np.mean])

In [ ]:
traces = []
barwidth_scalar = 1/4
idx = pd.IndexSlice

barwidth = mean.index * barwidth_scalar

mean = agged.loc[:, idx[:, "mean"]]
std = agged.loc[:, idx[:, "std"]]

# Collectl data
traces.append(Bar(
    x = mean.index,
    y = mean[("[CPU]Sys%", "mean")],
    yaxis = 'y2',
    name = 'CPU Sys%',
    width = barwidth,
    opacity=0.6
))
traces.append(Bar(
    x = mean.index,
    y = mean[("[CPU]Soft%", "mean")],
    yaxis = 'y2',
    name = 'CPU Soft%',
    width = barwidth,
    opacity=0.6
))
traces.append(Bar(
    x = mean.index,
    y = mean[("[CPU]User%", "mean")],
    yaxis = 'y2',
    name = 'CPU User%',
    width = barwidth,
    opacity=0.6
))

# Throughput
traces.append(Scatter(
    x = mean.index,
    y = mean[("throughput", "mean")],
    name = "throughput",
    error_y = dict(
        type='data',
        array=std[("throughput", "std")],
        visible=True
    ),
))
    
    
layout = Layout (
    xaxis = {'title': 'ncores', 'type': 'log'},
    yaxis = {'title': 'throughput (pps)', 'rangemode': 'tozero'},
    yaxis2 = {'title': 'CPU utilization', 
              'side': 'right', 
              'overlaying': 'y',
              'showgrid': False},
    title = 'Small-packet throughput',
    legend = {'x': 0.01, 'y': 1},
    barmode = "stack"
)

fig = Figure(data=traces, layout=layout)

py.iplot(fig, filename='tcp-cpu-throughput-varycores-2048cli')

In [40]:
std[("throughput", "std")]/mean[("throughput", "mean")]

ncores
8     0.004042
16    0.179658
32    0.200193
dtype: float64

In [43]:
data = []

i = 1
for label, df in grouped:
    data.append(Histogram(x=df["throughput"],
                          nbinsx=4,
                          name="%s cores" % label,
                          opacity=0.6,
                          histnorm="probability"))

layout = Layout (
    xaxis = {'title': 'throughput (pps)'},
    yaxis = {'title': 'frequency'},
    title = 'Throughput frequency for varying core counts',
    showlegend = True,
) 
fig = Figure(data=data, layout=layout)
    
py.iplot(fig, filename="tput_frequency_durations_ncores")